In [5]:
import pandas as pd

df = pd.read_csv('Generalinformationofschools.csv')

print(df.head())


                      school_name                            url_address  \
0        ADMIRALTY PRIMARY SCHOOL       https://admiraltypri.moe.edu.sg/   
1      ADMIRALTY SECONDARY SCHOOL     http://www.admiraltysec.moe.edu.sg   
2    AHMAD IBRAHIM PRIMARY SCHOOL  http://www.ahmadibrahimpri.moe.edu.sg   
3  AHMAD IBRAHIM SECONDARY SCHOOL  http://www.ahmadibrahimsec.moe.edu.sg   
4                  AI TONG SCHOOL           http://www.aitong.moe.edu.sg   

                   address  postal_code telephone_no telephone_no_2    fax_no  \
0    11   WOODLANDS CIRCLE       738907     63620598             na  63627512   
1  31   WOODLANDS CRESCENT       737916     63651733       63654596  63652774   
2    10   YISHUN STREET 11       768643     67592906             na  67592927   
3     751  YISHUN AVENUE 7       768928     67585384             na  67557778   
4   100  Bright Hill Drive       579646     64547672             na  64532726   

  fax_no_2            email_address                  mrt

In [9]:
addresses = set(df['address'].astype(str) + ' ' + df['postal_code'].astype(str))

print(addresses)

{'10   UBI AVENUE 1 408931', '150  Compassvale Bow 544822', '4    Chestnut Drive 679287', '12   MARSILING ROAD 739110', '1    Zubir Said Drive 227968', '3    ESSEX ROAD 309331', '10   SERANGOON AVENUE 4 556094', '600  UPPER EAST COAST ROAD 465561', '151  Towner Road 327830', '2    Tampines Ave 3 529706', '600  UPPER CHANGI ROAD 487012', '2    LORONG NAPIRI 547529', '50   BUKIT BATOK WEST AVE 8 658962', '3    ANG MO KIO AVE 10 569730', '2    BUKIT BATOK STREET 52 659243', '35   BUKIT BATOK STREET 31 659441', '3    BUKIT BATOK STREET 34 659322', '300  Tanglin Halt Road 148812', '41   EDGEFIELD PLAINS 828869', '49   MARINE CRESCENT 449761', '3    Tampines Street 22 529366', '30   JURONG WEST ST 61 648368', '21   TAMPINES STREET 71 529067', '61   HOUGANG AVENUE 8 538787', '50   CHOA CHU KANG NORTH 5 689621', '490  EAST COAST ROAD 429058', '307  BEDOK NORTH ROAD 469680', '111  Lorong Kismis 598112', '37   HILLCREST ROAD 288913', '6    ANG MO KIO STREET 22 569362', '1    HOUGANG ST 93 534238

In [10]:
len(addresses)

336

In [15]:
import requests
import json

def search(address):
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={address}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    response = requests.get(url)
    return response

def add_to_dict(response, address_to_coordinates, address):
    if response.status_code == 200:
        data = response.json()

        if data['found'] >= 1 and len(data['results']) >= 1:
            result = data['results']
            address_to_coordinates[address] = result
            return True 
    
    return False

def save_json(filename, data_dict):
    with open(filename, 'w') as json_file:
        json.dump(data_dict, json_file)

def read_json(filename):
    with open(filename, 'r') as json_file:
        loaded_data = json.load(json_file)
    return loaded_data



In [16]:
address_to_coordinates = {}

for i, address in enumerate(addresses):
    
    response = search(address)

    if not add_to_dict(response, address_to_coordinates, address):
        print(f"FAILED: {address}")
    
    if i % 100 == 0:
        save_json("address_to_coordinates.json", address_to_coordinates)
        print(f"{i}/{len(addresses)}")

save_json("address_to_coordinates.json", address_to_coordinates)


0/336
100/336
200/336
300/336


In [17]:
from pprint import pprint
test = read_json("address_to_coordinates.json")
pprint(test)

{'1    ANG MO KIO STREET 32 569299': [{'ADDRESS': '1 ANG MO KIO STREET 32 '
                                                  'NASCANS @ TECK GHEE '
                                                  'SINGAPORE 569299',
                                       'BLK_NO': '1',
                                       'BUILDING': 'NASCANS @ TECK GHEE',
                                       'LATITUDE': '1.36565018546903',
                                       'LONGITUDE': '103.851009800453',
                                       'POSTAL': '569299',
                                       'ROAD_NAME': 'ANG MO KIO STREET 32',
                                       'SEARCHVAL': 'NASCANS @ TECK GHEE',
                                       'X': '29968.822170008',
                                       'Y': '38632.1819182038'},
                                      {'ADDRESS': '1 ANG MO KIO STREET 32 TECK '
                                                  'GHEE PRIMARY SCHOOL '
                  

In [18]:
print(len(test))

336


In [24]:
import pandas as pd

df = pd.read_csv('Generalinformationofschools.csv')

address_to_coordinates = read_json("address_to_coordinates.json")

def get_latitude(address):
    return address_to_coordinates[address][0]["LATITUDE"]

def get_longitude(address):
    return address_to_coordinates[address][0]["LONGITUDE"]

df['postal_code'] = df['postal_code'].astype(str)

df['address'] = df['address'] + " " + df['postal_code']

df['latitude'] = df['address'].apply(get_latitude)
df['longitude'] = df['address'].apply(get_longitude)

# Save the DataFrame to a JSON file
schools_json = "schools.json"
df.to_json(schools_json, orient='records', lines=False)
print(f"Data has been saved to {schools_json}")

Data has been saved to schools.json


In [25]:
address_to_coordinates

{'10   UBI AVENUE 1 408931': [{'SEARCHVAL': 'AFTER SCHOOL @ MAHA BODHI SCHOOL',
   'BLK_NO': '10',
   'ROAD_NAME': 'UBI AVENUE 1',
   'BUILDING': 'AFTER SCHOOL @ MAHA BODHI SCHOOL',
   'ADDRESS': '10 UBI AVENUE 1 AFTER SCHOOL @ MAHA BODHI SCHOOL SINGAPORE 408931',
   'POSTAL': '408931',
   'X': '35566.4064952666',
   'Y': '34533.273462953',
   'LATITUDE': '1.32858023873554',
   'LONGITUDE': '103.901306904157'},
  {'SEARCHVAL': 'MAHA BODHI SCHOOL',
   'BLK_NO': '10',
   'ROAD_NAME': 'UBI AVENUE 1',
   'BUILDING': 'MAHA BODHI SCHOOL',
   'ADDRESS': '10 UBI AVENUE 1 MAHA BODHI SCHOOL SINGAPORE 408931',
   'POSTAL': '408931',
   'X': '35565.4979411988',
   'Y': '34521.1796176471',
   'LATITUDE': '1.32847086658384',
   'LONGITUDE': '103.901298737314'}],
 '150  Compassvale Bow 544822': [{'SEARCHVAL': 'PALM VIEW PRIMARY SCHOOL',
   'BLK_NO': '150',
   'ROAD_NAME': 'COMPASSVALE BOW',
   'BUILDING': 'PALM VIEW PRIMARY SCHOOL',
   'ADDRESS': '150 COMPASSVALE BOW PALM VIEW PRIMARY SCHOOL SINGAPOR